# Video Game Sales And Steam Video Games - Dataset Exploration and Cleaning

-- Ajit Mane (ajitmane36@gmail.com)

- Data Source
1. Video Game sales dataset from Kaggle:
https://www.kaggle.com/gregorut/videogamesales
2. Video Games traffic on steam dataset from Kaggle:
https://www.kaggle.com/tamber/steam-video-games

In [1]:
# Installing pyspark and findspark library
!pip install pyspark
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=a4e36afe13dc9abda11e244dfbb9224b893542bae49bd480e71292fad829fefc
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Importing necessary libraries
import pyspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import col, sum


In [3]:
# Creating instance of spark
spark = SparkSession.builder.appName("Data Exploration and Cleaning").getOrCreate()
spark

In [4]:
# Connecting to google drive for conecting dataset
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
# Loading datasets
video_game_sales_df_path='/content/drive/MyDrive/AImaBetter/9. AlmaX/AlmaX Jobs And Tracking/Almabetter AlmaX/IndiGG/vgsales.csv'
video_games_traffic_df_path='/content/drive/MyDrive/AImaBetter/9. AlmaX/AlmaX Jobs And Tracking/Almabetter AlmaX/IndiGG/steam-200k.csv'
vgsales_df = spark.read.csv(video_game_sales_df_path, header=True, inferSchema=True)
vgtraffic_df = spark.read.csv(video_games_traffic_df_path, header=True, inferSchema=True)

In [6]:
# Video games sales dataset
vgsales_df.show()

+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|Rank|                Name|Platform|Year|       Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|   1|          Wii Sports|     Wii|2006|      Sports|            Nintendo|   41.49|   29.02|    3.77|       8.46|       82.74|
|   2|   Super Mario Bros.|     NES|1985|    Platform|            Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|
|   3|      Mario Kart Wii|     Wii|2008|      Racing|            Nintendo|   15.85|   12.88|    3.79|       3.31|       35.82|
|   4|   Wii Sports Resort|     Wii|2009|      Sports|            Nintendo|   15.75|   11.01|    3.28|       2.96|        33.0|
|   5|Pokemon Red/Pokem...|      GB|1996|Role-Playing|            Nintendo|   11.27|    8.89|   10.22|  

In [7]:
# Video games traffic dataset
vgtraffic_df.show()

+---------+--------------------------+--------+-----+---+
|151603712|The Elder Scrolls V Skyrim|purchase|  1.0|  0|
+---------+--------------------------+--------+-----+---+
|151603712|      The Elder Scrolls...|    play|273.0|  0|
|151603712|                 Fallout 4|purchase|  1.0|  0|
|151603712|                 Fallout 4|    play| 87.0|  0|
|151603712|                     Spore|purchase|  1.0|  0|
|151603712|                     Spore|    play| 14.9|  0|
|151603712|         Fallout New Vegas|purchase|  1.0|  0|
|151603712|         Fallout New Vegas|    play| 12.1|  0|
|151603712|             Left 4 Dead 2|purchase|  1.0|  0|
|151603712|             Left 4 Dead 2|    play|  8.9|  0|
|151603712|                  HuniePop|purchase|  1.0|  0|
|151603712|                  HuniePop|    play|  8.5|  0|
|151603712|             Path of Exile|purchase|  1.0|  0|
|151603712|             Path of Exile|    play|  8.1|  0|
|151603712|               Poly Bridge|purchase|  1.0|  0|
|151603712|   

- Found that, first row values as header present in video games traffic dataset, so giving appropriate feature names to dataset

In [8]:
# Adding column names for video games traffic dataset
column_name=[ 'user-id', 'game-title', 'behavior-name', 'value','0']
vgtraffic_df = vgtraffic_df.toDF(*column_name)
vgtraffic_df.show()

+---------+--------------------+-------------+-----+---+
|  user-id|          game-title|behavior-name|value|  0|
+---------+--------------------+-------------+-----+---+
|151603712|The Elder Scrolls...|         play|273.0|  0|
|151603712|           Fallout 4|     purchase|  1.0|  0|
|151603712|           Fallout 4|         play| 87.0|  0|
|151603712|               Spore|     purchase|  1.0|  0|
|151603712|               Spore|         play| 14.9|  0|
|151603712|   Fallout New Vegas|     purchase|  1.0|  0|
|151603712|   Fallout New Vegas|         play| 12.1|  0|
|151603712|       Left 4 Dead 2|     purchase|  1.0|  0|
|151603712|       Left 4 Dead 2|         play|  8.9|  0|
|151603712|            HuniePop|     purchase|  1.0|  0|
|151603712|            HuniePop|         play|  8.5|  0|
|151603712|       Path of Exile|     purchase|  1.0|  0|
|151603712|       Path of Exile|         play|  8.1|  0|
|151603712|         Poly Bridge|     purchase|  1.0|  0|
|151603712|         Poly Bridge

## Data Inispection

In [9]:
# Checking number of rows and columns present in each dataset

# Check the number of rows and columns for vgsales_df
print(f"vgsales_df contains {vgsales_df.count()} rows and {len(vgsales_df.columns)} columns")

# Check the number of rows and columns for vgtraffic_df
print(f"vgtraffic_df contains {vgtraffic_df.count()} rows and {len(vgtraffic_df.columns)} columns")


vgsales_df contains 16598 rows and 11 columns
vgtraffic_df contains 199999 rows and 5 columns


In [10]:
# Checking schema of the datasets
vgsales_df.printSchema()
vgtraffic_df.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: double (nullable = true)
 |-- EU_Sales: double (nullable = true)
 |-- JP_Sales: double (nullable = true)
 |-- Other_Sales: double (nullable = true)
 |-- Global_Sales: double (nullable = true)

root
 |-- user-id: integer (nullable = true)
 |-- game-title: string (nullable = true)
 |-- behavior-name: string (nullable = true)
 |-- value: double (nullable = true)
 |-- 0: integer (nullable = true)



- Datasets having diffrent data type features like integer, string, and double.
- Feature Year can not be string, need to be converted to integer.

# Understanding Variables

In [11]:
# Checking categorical and numerical variables from each dataset

# Categorical and numerical variables in the "Video Game sales" dataset
vgsales_categorical_cols = []
vgsales_numerical_cols = []

for column, dtype in vgsales_df.dtypes:
    if dtype == "string":
        vgsales_categorical_cols.append(column)
    elif dtype in ["int","double"]:
        vgsales_numerical_cols.append(column)

print(f"{len(vgsales_categorical_cols)} categorical columns present in 'Video Game sales' dataset: {vgsales_categorical_cols}")
print(f"{len(vgsales_numerical_cols)} Numerical columns present in 'Video Game sales' dataset: {vgsales_numerical_cols}")


# Categorical and numerical variables in the "Video Games traffic on steam" dataset
vgtraffic_categorical_cols = []
vgtraffic_numerical_cols = []

for column, dtype in vgtraffic_df.dtypes:
    if dtype == "string":
        vgtraffic_categorical_cols.append(column)
    elif dtype in ["byte", "short", "int", "long", "float", "double"]:
        vgtraffic_numerical_cols.append(column)

print(f"{len(vgtraffic_categorical_cols)} categorical columns present in 'Video Games traffic data' dataset: {vgtraffic_categorical_cols}")
print(f"{len(vgtraffic_numerical_cols)} numerical columns present in 'Video Games traffic data' dataset: {vgtraffic_numerical_cols}")

5 categorical columns present in 'Video Game sales' dataset: ['Name', 'Platform', 'Year', 'Genre', 'Publisher']
6 Numerical columns present in 'Video Game sales' dataset: ['Rank', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
2 categorical columns present in 'Video Games traffic data' dataset: ['game-title', 'behavior-name']
3 numerical columns present in 'Video Games traffic data' dataset: ['user-id', 'value', '0']



- At first, we found that the video game sales dataset had year as a categorical variable, but year was wrongly assigned as a string data type, so further in data cleaning and preprocession, we will convert it to an integer.

# Data Cleaning and Preprocessing

[1] Handling Duplicated data

In [12]:
# Check for duplicates in the "Video Game sales" dataset
vgsales_duplicates = vgsales_df.dropDuplicates()
num_vgsales_duplicates = vgsales_duplicates.count()

if num_vgsales_duplicates > 0:
    print("Duplicates found in 'Video Game sales' dataset.")
    print("Number of duplicate rows:", num_vgsales_duplicates)
else:
    print("No duplicates found in 'Video Game sales' dataset.")

# Check for duplicates in the "Video Games traffic on steam" dataset
vgtraffic_duplicates = vgtraffic_df.dropDuplicates()
num_vgtraffic_duplicates = vgtraffic_duplicates.count()

if num_vgtraffic_duplicates > 0:
    print("Duplicates found in 'Video Games traffic on steam' dataset.")
    print("Number of duplicate rows:", num_vgtraffic_duplicates)
else:
    print("No duplicates found in 'Video Games traffic on steam' dataset.")


Duplicates found in 'Video Game sales' dataset.
Number of duplicate rows: 16598
Duplicates found in 'Video Games traffic on steam' dataset.
Number of duplicate rows: 199292


In [13]:
# Dropping duplicated

# Drop duplicates in the "Video Game sales" dataset
vgsales_duplicates_removed = vgsales_df.dropDuplicates()
num_vgsales_rows = vgsales_duplicates_removed.count()

print("Number of rows remaining after dropping duplicates in 'Video Game sales' dataset:", num_vgsales_rows)

# Drop duplicates in the "Video Games traffic on steam" dataset
vgtraffic_duplicates_removed = vgtraffic_df.dropDuplicates()
num_vgtraffic_rows = vgtraffic_duplicates_removed.count()

print("Number of rows remaining after dropping duplicates in 'Video Games traffic' dataset:", num_vgtraffic_rows)


Number of rows remaining after dropping duplicates in 'Video Game sales' dataset: 16598
Number of rows remaining after dropping duplicates in 'Video Games traffic' dataset: 199292


[2] Handling null/missing data

In [14]:
# Checking null values for each dataset

from pyspark.sql.functions import col, sum

# Calculate the number of nulls for each variable in the "Video Game sales" dataset
vgsales_null_counts = vgsales_duplicates_removed.select(*[sum(col(c).isNull().cast("int")).alias(c) for c in vgsales_duplicates_removed.columns])

# Calculate the number of nulls for each variable in the "Video Games traffic on steam" dataset
vgtraffic_null_counts = vgtraffic_duplicates_removed.select(*[sum(col(c).isNull().cast("int")).alias(c) for c in vgtraffic_duplicates_removed.columns])

# Create a table to display the variable name and number of nulls for the "Video Game sales" dataset
vgsales_null_table = spark.createDataFrame([(c, vgsales_null_counts.first()[c]) for c in vgsales_null_counts.columns], ["Variable", "Nulls"])

# Create a table to display the variable name and number of nulls for the "Video Games traffic on steam" dataset
vgtraffic_null_table = spark.createDataFrame([(c, vgtraffic_null_counts.first()[c]) for c in vgtraffic_null_counts.columns], ["Variable", "Nulls"])

# Show the null table for the "Video Game sales" dataset
print("Null values in 'Video Game sales' dataset:")
vgsales_null_table.show(truncate=False)

# Show the null table for the "Video Games traffic on steam" dataset
print("Null values in 'Video Games traffic on steam' dataset:")
vgtraffic_null_table.show(truncate=False)


Null values in 'Video Game sales' dataset:
+------------+-----+
|Variable    |Nulls|
+------------+-----+
|Rank        |0    |
|Name        |0    |
|Platform    |0    |
|Year        |0    |
|Genre       |0    |
|Publisher   |0    |
|NA_Sales    |0    |
|EU_Sales    |0    |
|JP_Sales    |0    |
|Other_Sales |0    |
|Global_Sales|0    |
+------------+-----+

Null values in 'Video Games traffic on steam' dataset:
+-------------+-----+
|Variable     |Nulls|
+-------------+-----+
|user-id      |0    |
|game-title   |0    |
|behavior-name|0    |
|value        |0    |
|0            |0    |
+-------------+-----+



- Both datasets, "Video Game sales" and "Video Games traffic" do not contain any null or missing values.

[3] Basic description of both dataset

In [15]:
# Display the schema information of the "Video Games traffic on steam" dataset
print("Schema information of 'Video Games traffic on steam' dataset:")
vgtraffic_duplicates_removed.printSchema()

# Display the schema information of the "Video Game sales" dataset
print("Schema information of 'Video Game sales' dataset:")
vgsales_duplicates_removed.printSchema()


Schema information of 'Video Games traffic on steam' dataset:
root
 |-- user-id: integer (nullable = true)
 |-- game-title: string (nullable = true)
 |-- behavior-name: string (nullable = true)
 |-- value: double (nullable = true)
 |-- 0: integer (nullable = true)

Schema information of 'Video Game sales' dataset:
root
 |-- Rank: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: double (nullable = true)
 |-- EU_Sales: double (nullable = true)
 |-- JP_Sales: double (nullable = true)
 |-- Other_Sales: double (nullable = true)
 |-- Global_Sales: double (nullable = true)



- The "0" column in the video games traffic dataset is useless and should be removed.
- The "Year" column in the video games sales dataset is wrongly assigned as a string and should be converted to an integer.

In [16]:
# Remove the "0" column from the traffic dataset
vgtraffic_duplicates_removed = vgtraffic_duplicates_removed.drop("0")

# Convert the "Year" column in the sales dataset from string to integer
vgsales_duplicates_removed = vgsales_duplicates_removed.withColumn("Year", vgsales_duplicates_removed["Year"].cast("integer"))


In [17]:
# Print the schema of the modified "Video Games traffic" dataset
print("Schema of 'Video Games traffic on steam' dataset after removing the '0' column:")
vgtraffic_duplicates_removed.printSchema()

# Print the schema of the modified "Video Game sales" dataset
print("Schema of 'Video Game sales' dataset after converting the 'Year' column to integer:")
vgsales_duplicates_removed.printSchema()


Schema of 'Video Games traffic on steam' dataset after removing the '0' column:
root
 |-- user-id: integer (nullable = true)
 |-- game-title: string (nullable = true)
 |-- behavior-name: string (nullable = true)
 |-- value: double (nullable = true)

Schema of 'Video Game sales' dataset after converting the 'Year' column to integer:
root
 |-- Rank: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: double (nullable = true)
 |-- EU_Sales: double (nullable = true)
 |-- JP_Sales: double (nullable = true)
 |-- Other_Sales: double (nullable = true)
 |-- Global_Sales: double (nullable = true)



[4] Merge datasets

In [18]:
# DataFrames vgsales_duplicates_removed and vgtraffic_duplicates_removed

# Merge the datasets based on common columns
merged_df = vgsales_duplicates_removed.join(
    vgtraffic_duplicates_removed,
    vgsales_duplicates_removed["Name"] == vgtraffic_duplicates_removed["game-title"],
    "inner"
)

# Show the merged DataFrame
merged_df.show()


+-----+--------------------+--------+----+--------+--------------------+--------+--------+--------+-----------+------------+---------+--------------------+-------------+------+
| Rank|                Name|Platform|Year|   Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|  user-id|          game-title|behavior-name| value|
+-----+--------------------+--------+----+--------+--------------------+--------+--------+--------+-----------+------------+---------+--------------------+-------------+------+
| 1071|Sid Meier's Civil...|      PC|2010|Strategy|Take-Two Interactive|    0.98|    0.52|     0.0|       0.19|        1.69| 53875128|Sid Meier's Civil...|         play|   5.9|
|15592|       Peggle Nights|      PC|2008|  Puzzle|        PopCap Games|     0.0|    0.01|     0.0|        0.0|        0.02| 11373749|       Peggle Nights|         play|  19.5|
|  302|       Left 4 Dead 2|    X360|2009| Shooter|     Electronic Arts|    2.67|    0.89|    0.05|       0.37|    

In [19]:
# Checking row and columns
num_rows = merged_df.count()
num_columns = len(merged_df.columns)

print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

Number of rows: 85114
Number of columns: 15


- At the end, after doing all the cleaning and transformation, the merged dataset had 85114 rows and 15 columns.







[5] Checking correlation between Viewers and Global_Sales

In [20]:
# Optionally, perform additional analysis or computations on the merged data
correlation = merged_df.select("Global_Sales", "value").corr("Global_Sales", "value")

# Output the merged data and correlation result
merged_df.show(10)
print("Correlation between Global Sales and Viewers:", correlation)


+-----+--------------------+--------+----+--------+--------------------+--------+--------+--------+-----------+------------+--------+--------------------+-------------+-----+
| Rank|                Name|Platform|Year|   Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales| user-id|          game-title|behavior-name|value|
+-----+--------------------+--------+----+--------+--------------------+--------+--------+--------+-----------+------------+--------+--------------------+-------------+-----+
| 1071|Sid Meier's Civil...|      PC|2010|Strategy|Take-Two Interactive|    0.98|    0.52|     0.0|       0.19|        1.69|53875128|Sid Meier's Civil...|         play|  5.9|
|15592|       Peggle Nights|      PC|2008|  Puzzle|        PopCap Games|     0.0|    0.01|     0.0|        0.0|        0.02|11373749|       Peggle Nights|         play| 19.5|
|  302|       Left 4 Dead 2|    X360|2009| Shooter|     Electronic Arts|    2.67|    0.89|    0.05|       0.37|        3.99|4

- Weak Positive Correlation:
  - There is a weak positive correlation between the Global Sales of video games and the number of Viewers. This suggests that as the viewership increases, there is a slight tendency for the global sales of video games to increase as well. However, the correlation is relatively low, indicating that there are other factors influencing video game sales apart from viewership.

- Limited Impact of Viewers on Sales:
  - The correlation coefficient of 0.0247 suggests that the influence of viewership on global sales is limited. While there is a positive relationship, the strength of the correlation is weak, indicating that viewership alone may not be a significant predictor of video game sales. Other factors such as marketing strategies, game quality, pricing, and platform availability likely play a more substantial role in driving sales.

# Thank You.